## Feature 가공하고 저장하기

In [1]:
import pandas as pd
import numpy as np

### 1. 원본 데이터 읽어오기
- `articles.parquet` : 상품 정보
- `customers.parquet` : 고객 (유저) 정보
- `transactions_train.csv` : 거래 데이터 (유저의 상품 구매 내역)

Kaggle H&M dataset을 이용하는 hopsworks-tutorial에서 제공하는 파일 그대로 사용.

In [2]:
articles_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/articles.parquet')
articles_df["article_id"] = articles_df["article_id"].astype(str)

articles_df.head(3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
customers_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/customers.parquet')

customers_df.head(3)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...


In [4]:
trans_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/transactions_train.parquet')[:1_000_000]

trans_df.head(3)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2


In [5]:
N_USERS = 25_000
customers_df.dropna(inplace=True, subset=["age"])
customer_subset_df = customers_df.sample(N_USERS, random_state=27)
trans_df = trans_df.merge(customer_subset_df["customer_id"])

print(f"Subset has {len(trans_df):,} transactions in total.")

Subset has 18,481 transactions in total.


In [6]:
customers_df.dropna(axis=1, inplace=True)
articles_df.dropna(axis=1, inplace=True)

In [7]:
trans_df["article_id"] = trans_df["article_id"].astype(str)
trans_df['t_dat'] = trans_df['t_dat'].apply(lambda x: pd.to_datetime(x))

### 2. 원본 데이터 가공

In [8]:
%%writefile transformations.py

import numpy as np

def month_sin(t_dat):
    month = t_dat.month - 1
    C = 2*np.pi/12
    return np.sin(month*C).item()

def month_cos(t_dat):
    month = t_dat.month - 1
    C = 2*np.pi/12
    return np.cos(month*C).item()

Overwriting transformations.py


In [9]:
from transformations import month_sin, month_cos

trans_df["month_sin"] = trans_df["t_dat"].apply(month_sin)
trans_df["month_cos"] = trans_df["t_dat"].apply(month_cos)

# convert python datetime object to epoch-mili
trans_df.t_dat = trans_df.t_dat.values.astype(np.int64)

In [16]:
merged_df = pd.merge(trans_df, customers_df[['customer_id', 'age']], on='customer_id', how='inner')
merged_df = pd.merge(merged_df, articles_df[['article_id', 'garment_group_name', 'index_group_name']], on='article_id', how='inner')


merged_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
3,1537401600000000000,00ffed0316ae807cb9439799d73cd61fd7f6479a0f9a19...,641741001,0.078203,2,-0.866025,-0.5,46.0,Shoes,Ladieswear
4,1537401600000000000,00ffed0316ae807cb9439799d73cd61fd7f6479a0f9a19...,685683001,0.023458,2,-0.866025,-0.5,46.0,Special Offers,Ladieswear


### 3. 가공한 feature를 postgresql에 저장하기

In [10]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# host = 'ssm-develop.db.sinsang.market'
# port = 3306
# username = 'dealicious'
# password = quote_plus('tlstkd12!@')
# database_name = 'dealicious'


host = '127.0.0.1'
port = 3306
username = 'root'
password = '1111'
database_name = 'dealicious'
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}')
conn = engine.connect()

print(conn)

In [18]:
# retrievals feature view
merged_df.to_sql(name='rec_retrievals', con=conn, if_exists='replace', chunksize=1000, method='multi', index=False)

18481

In [20]:
# articles feature view
articles_df.to_sql(name='rec_articles', con=conn, if_exists='replace', chunksize=1000, method='multi', index=False)

105542

In [11]:
customers_df.to_sql(name='rec_customers', con=conn, if_exists='replace', chunksize=1000, method='multi', index=False)

1356119